# Modelos a implementar

En esta situacion vamos a implementar algunos de los modelos de aprendizaje supervisado y comparar cuales quedan mejor adaptadas para el problema a partir de métricas y definiciones que se determinarán durante el desarrollo.

# 1. K-Vecinos Cercanos

In [2]:
#Importacion de las librerias necesarias
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn import metrics
from matplotlib.colors import ListedColormap

In [5]:
#Cargamos el dataframe procesado de nuestro repositorio de GitHub
url = 'https://raw.githubusercontent.com/Axumis/Parcial_Aprendizaje_Automatico/refs/heads/main/data/processed/dataset_con_riesgo.csv'
df = pd.read_csv(url)

#Mostramos el dataframe
df

,Temperatura_[°C],Humedad_relativa_[%],Precipitación_[mm],Velocidad_viento_[km/h],Riesgo
0,6.273000,63.0,0.0,20.780453,Alto
1,6.273000,64.0,0.0,20.380579,Alto
2,7.023000,60.0,0.0,22.530937,Alto
3,8.023000,55.0,0.0,27.193705,Muy alto
4,8.672999,53.0,0.0,26.400545,Alto
...,...,...,...,...,...
22022,-1.327000,84.0,0.0,3.545589,Moderado
22023,-1.827000,88.0,0.0,3.826853,Moderado
22024,-2.227000,91.0,0.0,4.104631,Moderado
22025,-2.177000,92.0,0.0,3.826853,Moderado
